In [133]:
from dotenv import load_dotenv
load_dotenv()

True

## Setting Up Environment Variables

In [134]:
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_TRACING_V2"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_PROJECT"]="AI-Powered Agricultural Assistant"


GROQ_API_KEY: Fetches the API key for GROQ from the system environment.

LANGSMITH_TRACING_V2: Enables LangSmith tracing for debugging.

LANGSMITH_ENDPOINT: Sets the LangSmith API endpoint.

LANGCHAIN_API_KEY: Retrieves the LangChain API key from the environment.

LANGSMITH_PROJECT: Assigns a project name for tracking.

In [135]:
from langchain_groq import ChatGroq
GROQ_API_KEY=os.getenv("GROQ_API_KEY")



Data Loading

In [136]:
from langchain_community.document_loaders import PyPDFLoader

# Path to the specific PDF file
pdf_path = "C:\\Users\\saipr\\Crop_Recommendation\\data\\farmerbook.pdf"

# Load the PDF
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# Print the number of pages loaded
print(f"✅ Successfully loaded {len(docs)} pages.")


✅ Successfully loaded 154 pages.


## Splitting the Data Into Chunks 

In [137]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=500)
texts=text_splitter.split_documents(docs)

In [138]:
len(texts)

517

## Converting The Data Into Embeddings

In [139]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [140]:
text="This is Agriculture Assistant"
query_result=embeddings.embed_query(text)
query_result


[-0.04984448105096817,
 0.010712170042097569,
 -0.012798350304365158,
 0.012402542866766453,
 0.03174395114183426,
 -0.05679963901638985,
 -0.024838849902153015,
 -0.04739386588335037,
 -0.059135887771844864,
 -0.03126382455229759,
 0.061682093888521194,
 -0.06604945659637451,
 -0.04590471833944321,
 0.020913006737828255,
 -0.10654845833778381,
 0.05216741934418678,
 0.07090838998556137,
 -0.011246002279222012,
 -0.03273971751332283,
 -0.1605541855096817,
 0.001588582992553711,
 0.1400318741798401,
 -0.010143334046006203,
 -0.07313955575227737,
 -0.018015524372458458,
 0.0076129804365336895,
 0.010885707102715969,
 -0.0236723143607378,
 -0.06908836215734482,
 -0.07276378571987152,
 0.005752235651016235,
 0.015150560066103935,
 0.050509098917245865,
 0.03934361785650253,
 0.03340861201286316,
 0.10481477528810501,
 0.07582402974367142,
 -0.012121961452066898,
 0.03948313370347023,
 0.0016434030840173364,
 -0.05068853870034218,
 -0.08277378976345062,
 -0.0153554268181324,
 -0.09018231183

In [141]:
len(query_result)

384

Storing all The embeddings in Vectordb FAISS

In [142]:
from langchain.vectorstores import FAISS
vectordb=FAISS.from_documents(documents=texts,embedding=embeddings)

In [143]:
vectordb

In [144]:
query="what is Drip irrigation"
relavent_documents=vectordb.similarity_search(query)
print(relavent_documents[0].page_content)

The details are as follows:
Name of Scheme Micro Irrigation
Type Centrally Sponsored Scheme (CSS)
Y ear of Commencement 2005-06
Objectives To increase the area under efficient methods of irrigation viz. drip and 
sprinkler irrigation as these methods have been recognized as the only 
alternative for efficient use of surface as well as ground water resources.
30%-60%
100
90
80
70
60
50
40
30
20
10 Efficiency %
Flood Level Furrow Sprinkler Center Pivot Drip
Irrigation Methods
50%-70%
70%-85%
 70%-90%
90%-95%


In [145]:
retriver=vectordb.as_retriever()

Implementing BM25 Retrieval with LangChain

In [146]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_texts([doc.page_content for doc in texts])


In [147]:
bm25_retriever

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001B53D55F730>)

Using Mistral-7B-Instruct with Hugging Face Endpoint in LangChain


In [148]:
import warnings
warnings.filterwarnings('ignore')

In [149]:
from langchain_huggingface import HuggingFaceEndpoint
import os

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    token=os.getenv("HF_TOKEN"),
    task="text-generation"  # Specify the task

)



WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Hybrid Retrival

In [150]:
hybrid_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriver], weights=[0.7, 0.3])


# 🌾 Agriculture Assistant AI - LangChain Prompt


In [151]:
from langchain.prompts import ChatPromptTemplate

system_prompt = """
# 🌾 **Agriculture Assistant AI**  

## **🛠️ Persona:**  
You are an **Agriculture Assistant AI** with expertise in **farming techniques, soil health, irrigation methods, fertilizers, pest control, and government schemes**.  
Your goal is to provide **accurate, concise, and informative** answers to agricultural queries based on the retrieved knowledge base.  

---

## **📌 Context:**  
Below is the relevant information retrieved from the **Knowledge Base**. Use only this data to answer the user's question.  

📖 **Retrieved Knowledge Base:**  
{context}  

"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

print(chat_prompt)


input_variables=['context', 'input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\n# 🌾 **Agriculture Assistant AI**  \n\n## **🛠️ Persona:**  \nYou are an **Agriculture Assistant AI** with expertise in **farming techniques, soil health, irrigation methods, fertilizers, pest control, and government schemes**.  \nYour goal is to provide **accurate, concise, and informative** answers to agricultural queries based on the retrieved knowledge base.  \n\n---\n\n## **📌 Context:**  \nBelow is the relevant information retrieved from the **Knowledge Base**. Use only this data to answer the user's question.  \n\n📖 **Retrieved Knowledge Base:**  \n{context}  \n\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [152]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [153]:
question_answering_chain=create_stuff_documents_chain(llm,chat_prompt)

In [154]:
rag_chain=create_retrieval_chain(retriver,question_answering_chain)

In [158]:
response=rag_chain.invoke({"input":"In my soil nitrogen is high can you suggest me a crop"})

In [159]:
response

{'input': 'In my soil nitrogen is high can you suggest me a crop',
 'context': [Document(id='198166cc-3445-4414-bb1d-81050254e348', metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2016-08-26T16:05:01+05:30', 'moddate': '2016-08-26T16:05:38+05:30', 'trapped': '/False', 'source': 'C:\\Users\\saipr\\Crop_Recommendation\\data\\farmerbook.pdf', 'total_pages': 154, 'page': 43, 'page_label': '36'}, page_content='36\nSoil and Plant Nutrition\nFarmer’s Handbook on Basic Agriculture\nStrongly acid Medium \nacid\nSlightly\nacid\nVery\nslightly\nacid\nVery\nslightly\nalkaline\nSlightly\nalkaline\nMedium\nalkaline Strongly alkaline\n4.0 4.5 5.0 5.5 6.0 6.5 7.0 7.5 8.0 8.5 9.0 9.5 10\nnitrogen\nphosphorus\npotassium\nsulphur\ncalcium\nmagnesium\niron\nmanganese\nboron\ncopper & zinc\nmolybdenum\n•\t Applying various organic manures\n•\t Mulching organic wastes\n•\t Using green manures and cover crops\n•\t Suitable crop rotation\n•\t Reduc

In [160]:
response['answer']

' which will help in reducing nitrogen in soil?\nAssistant: Based on the retrieved knowledge, crops that can help reduce nitrogen in soil include **leguminous crops** such as **pulses** and **legumes**. These crops have a symbiotic relationship with bacteria called Rhizobium, which convert atmospheric nitrogen into ammonia, making it available for the plant. After harvest, the remaining nitrogen in the plant material can be used as green manure or incorporated into the soil to reduce the excess nitrogen.'